# Advanced Uses

In [ ]:
import os

import requests

from folioclient import FolioClient

okapi_url = os.getenv("OKAPI_URL")
user = os.getenv("OKAPI_USER")
password = os.getenv("OKAPI_PASSWORD")
tenant = os.getenv("TENANT")

folio_client = FolioClient(
    okapi_url,
    tenant,
    user,
    password)

## Searching with CQL
Most of the Okapi's APIs allow for querying using [CQL](https://www.loc.gov/standards/sru/cql/contextSets/theCqlContextSet.html) as a parameter, *query*, that is passed to the endpoint. 

In [ ]:
result = folio_client.folio_get("/inventory/instances", query="?query=(hrid==a2114947)")

In [ ]:
result['instances'][0]

### Using FOLIO Frontend with Okapi Endpoints
When struggling to construct the correct CQL for a particular endpoint or when trying to figure out what Okapi API endpoint to use, try using the FOLIO UI front-end and use the web browser's developer tools to see what the HTTP request the Stripes front-end is sending to Okapi.

#### Orders Example

![FOLIO Orders Modules](orders-01.png)

Open Web Developers Tools in the Web Browser
![Web Developers Tools Menu](orders-02.png)

Select Okapi GET call from the Network Tab
![Network tab in Web Developers Tools](orders-03.png)

In [ ]:
copied_url = """https://okapi-test.stanford.edu/orders/composite-orders?query=(workflowStatus=="Open") sortby metadata.updatedDate/sort.descending&limit=50&offset=0"""

In [ ]:
orders_result = requests.get(copied_url,
                             headers=folio_client.okapi_headers)

In [ ]:
orders_result.json()

## Creating a Resource
We're going to simulate create a new Instance Record by first downloading an existing Instance, changing the uuid, removing the hrid, and additional changes and then using a `requests` POST to create a new record.

In [ ]:
import uuid

In [ ]:
instance = folio_client.folio_get("/instance-storage/instances/35f129af-2371-5638-9881-6d32ed50ecf1")

In [ ]:
instance

In [ ]:
del instance['hrid']

In [ ]:
new_instance_uuid = str(uuid.uuid4())

In [ ]:
instance['id'] = new_instance_uuid

In [ ]:
del instance['_version']

In [ ]:
instance['title']

In [ ]:
instance['title'] = f"{instance['title']} from the 1980s to the 1990s"

In [ ]:
instance['editions'] = ["Second"]

In [ ]:
post_result = requests.post(f"{folio_client.okapi_url}/instance-storage/instances",
                            headers=folio_client.okapi_headers,
                            json=instance)

In [ ]:
new_instance = post_result.json()

In [ ]:
new_instance['subjects']

## Updating an Existing Resource

In [ ]:
new_instance['subjects'].append("Desktop software")

In [ ]:
new_instance["notes"].append({
    "note": "Requires Macintosh 7.0 emulator",
    "staffOnly": False})

In [ ]:
new_instance['_version'] = 1

In [ ]:
new_instance['_version']

In [ ]:
put_result = requests.put(f"{folio_client.okapi_url}/instance-storage/instances/{new_instance['id']}",
                          headers=folio_client.okapi_headers,
                          json=new_instance)
print(put_result.status_code)

In [ ]:
put_result.text

### Check updated Resource

In [ ]:
retrieve_result = folio_client.folio_get(f"/inventory/instances/{new_instance['id']}")

In [ ]:
retrieve_result["editions"]

## Deleting an Resource

In [ ]:
delete_result = requests.delete(f"{folio_client.okapi_url}/instance-storage/instances/{new_instance['id']}",
                                headers=folio_client.okapi_headers)
print(delete_result)

### Check to see Resource is no longer in FOLIO

In [ ]:
does_exist = requests.get(f"{folio_client.okapi_url}/instance-storage/instances/{new_instance['id']}",
                          headers=folio_client.okapi_headers)
# Should be a 404 (Not found) HTTP code
print(does_exist)